<a href="https://colab.research.google.com/github/AbdNasir24/Atelier-2-NLP/blob/main/atelier4_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.naive_bayes import MultinomialNB

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Initialize spaCy lemmatizer and other tools
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


In [5]:
# Load the dataset
df = pd.read_csv('/answers.csv')


In [6]:
# Initialize spaCy lemmatizer and other tools
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


In [7]:
# Load the dataset
df = pd.read_csv('/answers.csv')


In [8]:
# Preprocessing function
def preprocess_text(text):
    if isinstance(text, float):
        return ""
    # Tokenization
    tokens = word_tokenize(text)

    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Removing stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization
    lemmatized_tokens = [token.lemma_ for token in nlp(" ".join(stemmed_tokens))]

    return " ".join(lemmatized_tokens)


In [13]:
# Convert 'correct' column to string type if not already
df['answer'] = df['answer'].astype(str)

# Remove rows where 'correct' is empty after conversion
df = df[df['answer'].str.strip() != ""]

# Apply preprocessing to the dataset
df['processed_text'] = df['answer'].apply(preprocess_text)

# Check for empty processed texts and filter them out
df = df[df['processed_text'].str.strip() != ""]


In [14]:
# Feature extraction
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(df['processed_text'])

# Ensure the target variable is numeric
df['score'] = pd.to_numeric(df['score'], errors='coerce')

# Drop rows with missing target values
df = df.dropna(subset=['score'])

# Ensure there are no empty rows in X_tfidf
non_empty_docs = np.sum(X_tfidf.toarray(), axis=1) > 0
X_tfidf = X_tfidf[non_empty_docs]
df = df.iloc[non_empty_docs]


In [15]:
# Regression Example
y = df['score']

# Split the data
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)

# Evaluate Linear Regression
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Metrics:")
print("Mean Squared Error:", mse_lr)
print("Root Mean Squared Error:", rmse_lr)
print("R2 Score:", r2_lr)


Linear Regression Metrics:
Mean Squared Error: 2.5865523262524026
Root Mean Squared Error: 1.6082761971292128
R2 Score: -1.1193250554250582


In [16]:
# Classification Example
def convert_to_category(score):
    if score <= 2:
        return 'low'
    elif score == 3:
        return 'medium'
    else:
        return 'high'

y_categorized = df['score'].apply(convert_to_category)

# Split the data
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y_categorized, test_size=0.2, random_state=42)

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)
y_pred_nb = nb_model.predict(X_test_tfidf)

# Evaluate Naive Bayes
print("Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))


Naive Bayes Classification Report:
              precision    recall  f1-score   support

        high       0.85      1.00      0.92       412
         low       1.00      0.05      0.09        43
      medium       0.00      0.00      0.00        32

    accuracy                           0.85       487
   macro avg       0.62      0.35      0.34       487
weighted avg       0.81      0.85      0.78       487



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
